In [1]:
import pandas as pd
import os
import joblib


In [2]:
df = pd.read_csv("assets\IMDB Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
df['review'] = df['review'].str.lower()
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


In [5]:
df.drop_duplicates(inplace=True)

In [6]:
import re
def rem_tags(text):
    p= re.compile('<.*?>')
    return p.sub(r'',text)

In [7]:
df['review'] = df['review'].apply(rem_tags)

In [8]:
from nltk.corpus import stopwords
st_words= set(stopwords.words('english'))
def rem_stopwords(text):
    words= text.split()
    fil_words= [word for word in words if word.lower() not in st_words]
    return " ".join(fil_words)

In [9]:
df['review']=df['review'].apply(rem_stopwords)

In [10]:
import string 
trash= string.punctuation

def rem_punc(text):
    tt= str.maketrans('','',trash)
    return text.translate(tt)

In [11]:
df['review'] = df['review'].apply(rem_punc)

In [12]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

stemmer= PorterStemmer()

df['tokenized_review']= df['review'].apply(lambda x: word_tokenize(x))
df['tokenized_review'] = df['tokenized_review'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

In [13]:
df.head()

,review,sentiment,tokenized_review
0,one reviewers mentioned watching 1 oz episode ...,positive,"[one, review, mention, watch, 1, oz, episod, h..."
1,wonderful little production filming technique ...,positive,"[wonder, littl, product, film, techniqu, unass..."
2,thought wonderful way spend time hot summer we...,positive,"[thought, wonder, way, spend, time, hot, summe..."
3,basically theres family little boy jake thinks...,negative,"[basic, there, famili, littl, boy, jake, think..."
4,petter matteis love time money visually stunni...,positive,"[petter, mattei, love, time, money, visual, st..."


In [14]:
df['joined_review'] =  df['tokenized_review'].apply(lambda x: ' '.join(x))
df

,review,sentiment,tokenized_review,joined_review
0,one reviewers mentioned watching 1 oz episode ...,positive,"[one, review, mention, watch, 1, oz, episod, h...",one review mention watch 1 oz episod hook righ...
1,wonderful little production filming technique ...,positive,"[wonder, littl, product, film, techniqu, unass...",wonder littl product film techniqu unassum old...
2,thought wonderful way spend time hot summer we...,positive,"[thought, wonder, way, spend, time, hot, summe...",thought wonder way spend time hot summer weeke...
3,basically theres family little boy jake thinks...,negative,"[basic, there, famili, littl, boy, jake, think...",basic there famili littl boy jake think there ...
4,petter matteis love time money visually stunni...,positive,"[petter, mattei, love, time, money, visual, st...",petter mattei love time money visual stun film...
...,...,...,...,...
49995,thought movie right good job creative original...,positive,"[thought, movi, right, good, job, creativ, ori...",thought movi right good job creativ origin fir...
49996,bad plot bad dialogue bad acting idiotic direc...,negative,"[bad, plot, bad, dialogu, bad, act, idiot, dir...",bad plot bad dialogu bad act idiot direct anno...
49997,catholic taught parochial elementary schools n...,negative,"[cathol, taught, parochi, elementari, school, ...",cathol taught parochi elementari school nun ta...
49998,im going disagree previous comment side maltin...,negative,"[im, go, disagre, previou, comment, side, malt...",im go disagre previou comment side maltin one ...


In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df['joined_review'],df['sentiment'],test_size=0.3,stratify=df['sentiment'],random_state=23)

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [17]:
pipeline = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('nbclf', MultinomialNB())
                    ])

In [18]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('nbclf', MultinomialNB())])

In [19]:
y_pred= pipeline.predict(x_test)

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.86      0.87      0.86      7410
    positive       0.87      0.85      0.86      7465

    accuracy                           0.86     14875
   macro avg       0.86      0.86      0.86     14875
weighted avg       0.86      0.86      0.86     14875



In [21]:
joblib.dump(pipeline, 'sentiment_analysis.pkl')

['sentiment_analysis.pkl']

In [22]:
yt = pd.read_csv("assets\youtube_comments2.csv")
yt

,comment,likes
0,Consistent carriapa is my fav genre,198
1,NN EP IS AN ABSOLUTE VIBE ️,253
2,Hell yeahh a full fledged video on emiway ep,192
3,11:43 KSHAMA EP,62
4,Bkp don't hate anyone about his view/critics ...,105
...,...,...
307,Emiway \nAs a singer \nas a rapper,0
308,Umm i don't know but i feel emiways voice soun...,0
309,Beef ke baad ye sab kyu banjate Arijit,0
310,Femiway chhake ke gaane kon kon sunta h like k...,0


In [23]:
yt.drop(columns=['likes'],inplace=True)
yt

,comment
0,Consistent carriapa is my fav genre
1,NN EP IS AN ABSOLUTE VIBE ️
2,Hell yeahh a full fledged video on emiway ep
3,11:43 KSHAMA EP
4,Bkp don't hate anyone about his view/critics ...
...,...
307,Emiway \nAs a singer \nas a rapper
308,Umm i don't know but i feel emiways voice soun...
309,Beef ke baad ye sab kyu banjate Arijit
310,Femiway chhake ke gaane kon kon sunta h like k...


In [24]:
yt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  310 non-null    object
dtypes: object(1)
memory usage: 2.6+ KB


In [25]:
yt['comment'] = yt['comment'].fillna('')

In [26]:

y_pred= pipeline.predict(yt['comment'])

In [27]:
print(y_pred)

['positive' 'positive' 'positive' 'positive' 'positive' 'negative'
 'negative' 'positive' 'positive' 'positive' 'positive' 'negative'
 'positive' 'positive' 'positive' 'negative' 'positive' 'positive'
 'positive' 'negative' 'positive' 'positive' 'negative' 'positive'
 'negative' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'negative' 'negative' 'positive' 'positive' 'positive'
 'positive' 'positive' 'negative' 'negative' 'positive' 'positive'
 'positive' 'positive' 'negative' 'positive' 'negative' 'negative'
 'positive' 'negative' 'positive' 'positive' 'positive' 'positive'
 'positive' 'negative' 'positive' 'positive' 'negative' 'negative'
 'negative' 'positive' 'positive' 'positive' 'negative' 'positive'
 'negative' 'positive' 'positive' 'negative' 'positive' 'negative'
 'positive' 'negative' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'negative' 'positive' 'negative'
 'positive' 'negative' 'positive' 'negative' 'positive' 'negat

In [28]:
comments = []
predictions = []


for i in yt['comment']:
    y_pred = pipeline.predict([i])  
    comments.append(i)
    predictions.append(y_pred[0])  


result_df = pd.DataFrame({
    'comment': comments,
    'predicted_sentiment': predictions
})

In [29]:
result_df

,comment,predicted_sentiment
0,Consistent carriapa is my fav genre,positive
1,NN EP IS AN ABSOLUTE VIBE ️,positive
2,Hell yeahh a full fledged video on emiway ep,positive
3,11:43 KSHAMA EP,positive
4,Bkp don't hate anyone about his view/critics ...,positive
...,...,...
307,Emiway \nAs a singer \nas a rapper,negative
308,Umm i don't know but i feel emiways voice soun...,negative
309,Beef ke baad ye sab kyu banjate Arijit,positive
310,Femiway chhake ke gaane kon kon sunta h like k...,positive
